In [1]:
from jupyter_dash import JupyterDash  
from dash import dcc, html

In [3]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash
from dash import dcc, html
app = JupyterDash(__name__)

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
from bson import ObjectId
import json

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AnimalShelterCRUD import AnimalShelter

###########################
# Data Manipulation / Model
###########################
#username = "aacuser"
#password = "SNHU2025"

# Connect to database via CRUD Module
shelter = AnimalShelter()

try:
    # Pull all documents
    data = shelter.read({})
    print("Connected! Records found:", len(data))

    # Convert to DataFrame
    df = pd.DataFrame.from_records(data)

    # Drop _id only if it exists
    if '_id' in df.columns:
        df.drop(columns=['_id'], inplace=True)

    # Preview first few rows
    display(df.head())

except Exception as e:
    print("Error:", e)

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date


app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('CS-340 Dashboard - Issaih Perez'))), #Unique Identifier
    html.Center(html.A(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())), href='http://www.snhu.edu')),
    html.Hr(), #When the above image is clicked, it brings the user to the listed website (www.snhu.edu)
    html.Div([ #FIXME Add in code for the interactive filtering options. 
        #For example, Radio buttons, drop down, checkboxes, etc.
        dcc.Dropdown(
            id='filter-type',
            options=[ #List the filter types for the dashboard
                {'label': 'Disaster Rescue', 'value': 'disaster'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'mount'},
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Reset Filters', 'value': 'reset'},
                {'label': 'Dog', 'value': 'Dog'},
                {'label': 'Cat', 'value': 'Cat'},
            ],
            #value='reset',
            #labelStyle={'display': 'inline-block'}
            placeholder="Select a filter",
            clearable=True #Allows user to remove selection
        )
    ]),  
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        editable = False, 
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = "single",
        row_deletable = False,
        selected_columns = [],
        selected_rows = [0],
        page_action = "native",
        page_current = 0,
        page_size = 10
        
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    # Disaster Rescue
    if filter_type == 'disaster':
        df = pd.DataFrame.from_records(shelter.read({
            "animal_type": "Dog",
            "breed": {"$in": ["Doberman Pinscher", "German Shepard", "Golden Retriever",
                             "Bloodhound", "Rottweiler"
                             ]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte":20.0, "$lte":300.0}
        }))    
        
    # Mountain/Wilderness Rescue
    elif filter_type == 'mount':
        df = pd.DataFrame.from_records(shelter.read({
            "animal_type": "Dog",
            "breed": {"$in": ["German Shepard", "Alaskan Malamute", "Old English Sheepdog",
                              "Siberian Husky", "Rottweiler"
                             ]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
        }))  
    
    # Water Rescue
    elif filter_type == 'water':
        df = pd.DataFrame.from_records(shelter.read({
            "animal_type": "Dog",
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever",
                              "Newfoundland"
                             ]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
        }))  

    # Dog Filter
    elif filter_type == 'Dog':
        df = pd.DataFrame.from_records(shelter.read({
            "animal_type": "Dog"}))

    # Cat Filter
    elif filter_type == 'Cat':
        df = pd.DataFrame.from_records(shelter.read({
            "animal_type": "Cat"}))
    
    #Reset to default settings/No Filter
    else:
        query={}
        records=shelter.read(query)
        df=pd.DataFrame.from_records(records)
        #print("Records Fetched:", len(records)) #Debug statement
        #print(records)
        #print(df.head()) #prints out the first couple rows of the database
    df = df.drop_duplicates(subset='animal_id', keep='first') # Deletes any duplicate ID's 
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    df.drop(columns=['_id'], inplace=True)
    def is_serializable(row): # Makes everything json serializable to avoid any null value types
        try:
            json.dumps(row)
            return True
        except(TypeError, OverflowError):
            return False
    data = df.to_dict("records")  
    filtered_data=[row for row in data if is_serializable(row)]
    return filtered_data
    

# Display the breeds of animal based on quantity represented in
# the data table

@app.callback(
    Output("breed-bar-chart", "figure"),
    Input("animal-type-dropdown", "value")
)
def update_bar_chart(animal_type):
    # Query Mongo based on filter
    if animal_type == "all":
        records = shelter.read({})
    else:
        records = shelter.read({"animal_type": animal_type})

    df = pd.DataFrame(records)

    if df.empty:
        return px.bar(title="No data available")

    # Count by breed, top 10
    breed_counts = df["breed"].value_counts().nlargest(10).reset_index()
    breed_counts.columns = ["breed", "count"]

    fig = px.bar(
        breed_counts,
        x="breed",
        y="count",
        title=f"Top Breeds ({animal_type if animal_type != 'all' else 'All Types'})",
        text_auto=True
    )
    fig.update_layout(xaxis_title="Breed", yaxis_title="Count")
    return fig

   
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index): 
    # First check if viewData is None, if yes, the table has not been loaded yet, just display the map
    if viewData == None:
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'},
            center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id")]
                  )]                
    else:
        dff = pd.DataFrame.from_dict(viewData)
        if index is None:
            row = 0
        else: 
            row = index[0]
        # Check to see if the table is initialized as empty
        if not isinstance(viewData, list):
            return [html.H1(f"viewData is of type {type(viewData)}."), html.H1(f"index is of type {type(index)}.")]
        elif viewData == []:
            return [html.H1("viewData is empty. There is no table data")]
    
# Austin, TX is at [30.75, -97.48]
    for row in range(len(dff)): 
        lat = dff.iloc[row, 13]
        lon = dff.iloc[row, 14]
        #print(f"Row {row}: Latitude={lat}, Longitude={lon}") Debug statement to find the variables lat/long
    mask = dff.iloc[:, 13].notna() & dff.iloc[:, 14].notna()
    dff = dff[mask]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
              center=[30.75, -97.48], zoom=10, children=[
                  dl.TileLayer(id="base-layer-id"),
                  *[
                      dl.Marker(
                          position=[dff.iloc[row, 13], dff.iloc[row, 14]],
                          children=[
                              dl.Tooltip(dff.iloc[row, 4]),
                              dl.Popup([
                                  html.H1("Animal Name"),
                                  html.P(dff.iloc[row, 9])
                              ])
                          ]
                      )for row in range(len(dff))
                  ]
              ]
              )
    ]

Connected! Records found: 0


C:\CS340_Project\.venv\Lib\site-packages\dash\dash.py:634: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



""


In [4]:
app.run_server(mode="inline", debug=True)

#if __name__ == "__main__":      Only use if transitioning into Dash instead of JupyterDash
    #app.run_server(debug=True)

AttributeError: 'super' object has no attribute 'run_server'